In [2]:
#CLIMATE CHANGE TIME SERIES ANALYSIS

#Importing Liabraries
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#importing data
df = pd.read_csv(r"C:\Users\HP\Documents\CV\GlobalLandTemperaturesByCity.csv")

In [4]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [5]:
#Exploring data types
df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [6]:
df.shape

(1048575, 7)

In [7]:
df.isnull().sum()

dt                                   0
AverageTemperature               47547
AverageTemperatureUncertainty    47547
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [8]:
#drop all null values
df=df.dropna(how= 'any',axis=0)

In [9]:
df.shape

(1001028, 7)

In [11]:
#rename columns
df.rename(columns={'dt' : 'Date','AverageTemperatureUncertainty': 'confidence_interval_temp'},inplace=True)
df.head()

,Date,AverageTemperature,confidence_interval_temp,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E


In [12]:
#Data Preparation. 
#converting Date time column to "Date time" datatype
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.index

DatetimeIndex(['1743-11-01', '1744-04-01', '1744-05-01', '1744-06-01',
               '1744-07-01', '1744-09-01', '1744-10-01', '1744-11-01',
               '1744-12-01', '1745-01-01',
               ...
               '1783-11-01', '1783-12-01', '1784-01-01', '1784-02-01',
               '1784-03-01', '1784-04-01', '1784-05-01', '1784-06-01',
               '1784-07-01', '1784-08-01'],
              dtype='datetime64[ns]', name='Date', length=1001028, freq=None)

In [13]:
df.describe()

,AverageTemperature,confidence_interval_temp
count,1.001028e+06,1.001028e+06
mean,1.792899e+01,1.033831e+00
std,1.035884e+01,1.103003e+00
min,-3.187400e+01,3.600000e-02
25%,1.201700e+01,3.480000e-01
50%,2.048600e+01,6.070000e-01
75%,2.592600e+01,1.380000e+00
max,3.915600e+01,1.503000e+01


In [14]:
#Now we use year as an index
df['year']= df.index.year
df.head()

,AverageTemperature,confidence_interval_temp,City,Country,Latitude,Longitude,year
Date,,,,,,,
1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E,1743
1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E,1744
1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E,1744
1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E,1744
1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E,1744


In [15]:
#Filtering to use latest data
latest_df= df.loc['1980': '2021']

In [16]:
latest_df.head()

,AverageTemperature,confidence_interval_temp,City,Country,Latitude,Longitude,year
Date,,,,,,,
1980-01-01,-1.850,0.363,Århus,Denmark,57.05N,10.33E,1980
1980-01-02,-2.171,0.377,Århus,Denmark,57.05N,10.33E,1980
1980-01-03,0.625,0.178,Århus,Denmark,57.05N,10.33E,1980
1980-01-04,6.151,0.203,Århus,Denmark,57.05N,10.33E,1980
1980-01-05,11.434,0.226,Århus,Denmark,57.05N,10.33E,1980


In [17]:
latest_df.describe()

,AverageTemperature,confidence_interval_temp,year
count,170519.000000,170519.000000,170519.000000
mean,19.364059,0.365391,1996.339663
std,9.966120,0.196611,9.721861
min,-28.973000,0.036000,1980.000000
25%,14.092000,0.241000,1988.000000
50%,22.245000,0.320000,1996.000000
75%,26.740000,0.435000,2005.000000
max,39.156000,5.148000,2013.000000


In [18]:
#Grouping data countrywise based on avergae temperature
latest_df[['Country','AverageTemperature']].groupby(['Country']).mean().sort_values('AverageTemperature')

,AverageTemperature
Country,
Norway,2.581287
Russia,4.224789
Kazakhstan,6.503722
Canada,7.376665
Switzerland,7.703418
...,...
Benin,27.665557
Guinea Bissau,27.729134
Thailand,27.765322


In [ ]:
#Plot graph between year and average temperature
plt.figure(figsize=(9,4))
sns.lineplot(x = "year" , y = "AverageTemperature",data=latest_df)
plt.show()